In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

In [ ]:
from src.oracle import generate_oracle_csv

generate_oracle_csv('example_oracle.txt', '../../AllPrintingsCSVFiles/cards.csv', 'output/oracle_output.csv')

Oracle CSV generated: output/oracle_output.csv


In [ ]:
# Load the data files
oracle_df = pd.read_csv('output/oracle_output.csv')
legacy_oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
print(f"Loaded {len(oracle_df)} cards from oracle_df")
print(f"Columns available: {list(oracle_df.columns)}")
oracle_df.head()

Loaded 450 cards from oracle_df
Columns available: ['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID', 'Power', 'Toughness']


,name,CMC,Type,Color,Color Category,Oracle Text,tags,MTGO ID,Power,Toughness
0,Boros Elite,1,Creature — Human Soldier,W,White,Battalion — Whenever this creature and at leas...,NaN,NaN,1.0,1.0
1,Deftblade Elite,1,Creature — Human Soldier,W,White,"Provoke (Whenever this creature attacks, you m...",NaN,NaN,1.0,1.0
2,Doomed Traveler,1,Creature — Human Soldier,W,White,"When this creature dies, create a 1/1 white Sp...",NaN,NaN,1.0,1.0
3,Elite Vanguard,1,Creature — Human Soldier,W,White,NaN,NaN,NaN,2.0,1.0
4,Faerie Guidemother,1,Creature — Faerie,W,White,Flying,NaN,NaN,1.0,1.0


# Theme Validation
Let's check if we have enough cards available for each theme in our jumpstart cube.

# Deck Construction
Now let's test the deck construction function to build actual jumpstart decks from our themes.

In [15]:
# Test the refactored deck construction function

# Build all jumpstart decks using the new refactored version
from src.construct import construct_jumpstart_decks, print_detailed_deck_analysis, CardConstraints, analyze_deck_composition

# Create constraints with custom target deck size and non-land limits
constraints = CardConstraints(
    target_deck_size=13,
    max_lands_dual=1,
    max_lands_mono=1,
    total_non_land=12,  # All 12 cards should be non-lands
    min_creatures=2,  # Set minimum to 4 for testing
    max_creatures=9
)

print("🚀 Starting deck construction with refactored algorithm...")
deck_dataframes = construct_jumpstart_decks(oracle_df, constraints=constraints)
legacy_deck_dataframes = construct_jumpstart_decks(legacy_oracle_df, constraints=constraints)

# Generate analysis first, then print detailed analysis
analysis = analyze_deck_composition(deck_dataframes)
print_detailed_deck_analysis(deck_dataframes, analysis, constraints)

🚀 Starting deck construction with refactored algorithm...
🏗️ CONSTRUCTING JUMPSTART DECKS

🔒 Phase 0: Core card reservation
Ensuring each theme gets its defining cards before general competition...

🎯 White Soldiers: Reserving core cards
  ✅ Boros Elite               |   5.3 pts | Creature — Human Sol...
  ✅ Miner's Guidewing         |   5.3 pts | Creature — Bird
  ✅ Topan Freeblade           |   4.8 pts | Creature — Human Sol...
  ✅ Ainok Bond-Kin            |   4.3 pts | Creature — Dog Soldi...
  📦 Reserved 4 core cards

🎯 White Equipment: Reserving core cards
  ✅ Ancestral Blade           |  11.8 pts | Artifact — Equipment
  ✅ Vulshok Morningstar       |  11.8 pts | Artifact — Equipment
  ✅ Mandibular Kite           |  11.2 pts | Artifact — Equipment
  ✅ Greatsword of Tyr         |  11.2 pts | Artifact — Equipment
  ✅ Flayer Husk               |  11.2 pts | Artifact — Equipment
  📦 Reserved 5 core cards

🎯 White Angels: Reserving core cards
  ✅ Combat Professor          |   6.8 pts 

In [ ]:
from src.export import export_cube_to_csv


export_cube_to_csv(deck_dataframes, 'jumpstart_decks.csv', oracle_df)

Exporting jumpstart cube to jumpstart_decks.csv...
Adding 67 unassigned cards...
✅ Successfully exported 450 cards to jumpstart_decks.csv

📊 Export Summary:
Total cards: 450
Assigned cards: 383
Unassigned cards: 67
Number of decks: 30

Deck breakdown:
  White Equipment: 13 cards
  Simic Ramp Control: 13 cards
  Boros Aggro: 13 cards
  Golgari Graveyard Value: 13 cards
  Izzet Spells Matter: 13 cards
  Orzhov Lifegain Value: 13 cards
  Selesnya Value: 13 cards
  Gruul Midrange: 13 cards
  Rakdos Aggro: 13 cards
  Dimir Mill: 13 cards
  Unassigned: 67 cards
  ... and 20 more decks


'jumpstart_decks.csv'

In [ ]:
# Import validation functions and run card uniqueness validation
from src.validation import validate_card_uniqueness, validate_deck_constraints, validate_jumpstart_cube, display_validation_summary

# Run the validation
# validation_result = validate_card_uniqueness(deck_dataframes)
validate_jumpstart_cube(deck_dataframes, oracle_df, ALL_THEMES, constraints)

🎯 COMPREHENSIVE JUMPSTART CUBE VALIDATION
🔍 VALIDATING CARD UNIQUENESS
📊 VALIDATION RESULTS:
Total cards across all decks: 383
Unique cards used: 383
Duplicate cards found: 0

✅ VALIDATION PASSED!
All 383 cards are used exactly once.
🔍 VALIDATING DECK CONSTRAINTS
📊 CONSTRAINT VALIDATION RESULTS:
Valid decks: 23/30
Constraint violations: 7

❌ CONSTRAINT VIOLATIONS:
  Black Control:
    - Wrong deck size: 12/13
  Red Burn:
    - Wrong deck size: 12/13
  Red Dragons:
    - Wrong deck size: 12/13
  Red Artifacts:
    - Wrong deck size: 12/13
  Green Ramp:
    - Wrong deck size: 12/13
  Green Stompy:
    - Wrong deck size: 12/13
  Green Beasts:
    - Wrong deck size: 12/13

📈 CARD DISTRIBUTION ANALYSIS
📊 OVERALL STATISTICS:
Total cards available: 450
Total cards used: 383
Cards unused: 67
Usage rate: 85.1%

🎨 USAGE BY COLOR:
  White    :  63/ 68 cards ( 92.6%)
  Blue     :  63/ 68 cards ( 92.6%)
  Black    :  58/ 68 cards ( 85.3%)
  Red      :  62/ 67 cards ( 92.5%)
  Green    :  61/ 68 car

{'overall_valid': False,
 'uniqueness': {'valid': True,
  'total_cards': 383,
  'unique_cards': 383,
  'duplicates': {},
  'duplicate_count': 0,
  'extra_instances': 0},
 'constraints': {'valid': False,
  'valid_decks': 23,
  'total_decks': 30,
  'violations': [{'theme': 'Black Control',
    'violations': ['Wrong deck size: 12/13'],
    'creatures': 5,
    'unique_lands': 0,
    'deck_size': 12},
   {'theme': 'Red Burn',
    'violations': ['Wrong deck size: 12/13'],
    'creatures': 4,
    'unique_lands': 0,
    'deck_size': 12},
   {'theme': 'Red Dragons',
    'violations': ['Wrong deck size: 12/13'],
    'creatures': 9,
    'unique_lands': 0,
    'deck_size': 12},
   {'theme': 'Red Artifacts',
    'violations': ['Wrong deck size: 12/13'],
    'creatures': 7,
    'unique_lands': 0,
    'deck_size': 12},
   {'theme': 'Green Ramp',
    'violations': ['Wrong deck size: 12/13'],
    'creatures': 6,
    'unique_lands': 0,
    'deck_size': 12},
   {'theme': 'Green Stompy',
    'violations':

In [ ]:
# Additional analysis using the imported validation functions
from src.validation import analyze_card_distribution

# Run the distribution analysis
distribution_analysis = analyze_card_distribution(deck_dataframes, oracle_df, constraints)


📈 CARD DISTRIBUTION ANALYSIS
📊 OVERALL STATISTICS:
Total cards available: 450
Total cards used: 383
Cards unused: 67
Usage rate: 85.1%

🎨 USAGE BY COLOR:
  White    :  63/ 68 cards ( 92.6%)
  Blue     :  63/ 68 cards ( 92.6%)
  Black    :  58/ 68 cards ( 85.3%)
  Red      :  62/ 67 cards ( 92.5%)
  Green    :  61/ 68 cards ( 89.7%)
  Colorless:  30/ 30 cards (100.0%)

🎯 DECK COMPLETENESS:
Complete decks (13 cards): 23
Incomplete decks: 7

Incomplete deck details:
  Black Control: 12/13 cards
  Red Burn: 12/13 cards
  Red Dragons: 12/13 cards
  Red Artifacts: 12/13 cards
  Green Ramp: 12/13 cards
  Green Stompy: 12/13 cards
  Green Beasts: 12/13 cards

📋 UNUSED CARDS ANALYSIS:
Unused creatures: 11
Unused lands: 30
Unused spells: 26

Sample unused cards:
  • Alabaster Host Intercessor (Creature — Phyrexian Samurai) - W
  • Borrowed Grace (Instant) - W
  • Settle Beyond Reality (Sorcery) - W
  • Oblivion Ring (Enchantment) - W
  • Faith's Fetters (Enchantment — Aura) - W
  • Sword Coast 

In [ ]:
# Radar chart for a sample of decks (requires plotly)
import plotly.graph_objects as go
sample_themes = metrics_df.index[:]  # Show first 5 decks as example
radar_metrics = ['avg_cmc', 'card_quality', 'archetype_alignment', 'synergy', 'keyword_density']
fig = go.Figure()
for theme in sample_themes:
    values = metrics_df.loc[theme, radar_metrics].values.tolist()
    fig.add_trace(go.Scatterpolar(
        r=values + [values[0]],
        theta=radar_metrics + [radar_metrics[0]],
        fill='toself',
        name=theme
    ))
fig.update_layout(
    polar=dict(radialaxis=dict(visible=True)),
    showlegend=True,
    title='Deck Balance Radar Chart (Sample)'
)
fig.show()

NameError: name 'metrics_df' is not defined